In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

ckpt_converter = '../tts/openvoice_models_downloaded/checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')


os.makedirs(output_dir, exist_ok=True)

/opt/homebrew/Caskroom/miniconda/base/envs/MUC_py311new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



/opt/homebrew/Caskroom/miniconda/base/envs/MUC_py311new/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint '../tts/openvoice_models_downloaded/checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


ERROR: Could not find a version that satisfies the requirement openvoice (from versions: none)
ERROR: No matching distribution found for openvoice
Note: you may need to restart the kernel to use updated packages.


In [2]:
reference_speaker = './Corian Recording.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v2
[(3.534, 62.706), (62.894, 121.298), (122.798, 124.242), (126.158, 126.7960625)]
after vad: dur = 119.6580045351474
min value is  tensor(-1.1720)
max value is  tensor(1.1780)


/opt/homebrew/Caskroom/miniconda/base/envs/MUC_py311new/lib/python3.11/site-packages/torch/functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/SpectralOps.cpp:880.)
  return _VF.stft(  # type: ignore[attr-defined]


min value is  tensor(-1.2770)
max value is  tensor(1.2928)
min value is  tensor(-1.2272)
max value is  tensor(1.2175)
min value is  tensor(-1.1123)
max value is  tensor(1.1186)
min value is  tensor(-1.2032)
max value is  tensor(1.1556)
min value is  tensor(-1.1417)
max value is  tensor(1.1840)
min value is  tensor(-1.1973)
max value is  tensor(1.1610)
min value is  tensor(-1.1373)
max value is  tensor(1.1265)
min value is  tensor(-1.1333)
max value is  tensor(1.1505)


In [3]:
audio_name

'Corian Recording_v2_N3JlMPm8BhO0ssLy'

In [23]:
from melo.api import TTS

long_text = """
So, umm, I was thinking about what makes a voice truly human. It's not just the words, right? Like, if I say, 'Hmm, that's an interesting point,' that 'hmm' carries a lot of meaning. It's a thoughtful pause, a moment of consideration.

And then there are the fillers. You know, like, 'We need to, uhh, re-evaluate our strategy.' That 'uhh' isn't just wasted breath; it often signals that the speaker is searching for the right words, or maybe slightly uncertain.

Sometimes it's a quick intake of breath, a little 'agh' of surprise or mild annoyance. Or a sigh – a deep 'aaah,' conveying relief or perhaps exasperation. It's subtle, but we interpret it instantly.

Even a simple 'oh!' can be, well, varied. Is it 'Oh, really?' with rising surprise, or 'Oh, no...' with a falling, disappointed tone? These tiny vocalizations, the pitch shifts, the slight hesitations, the emphasis on certain syllables... they're the glue that gives our conversations texture and, uhm, personality. A lot of personality, actually.

And let's not forget the expressive sounds – a quick 'shhh' for silence, or a guttural 'ugh' of frustration. A genuine laugh, or a little, almost unconscious, 'hehe' when something's mildly amusing. These aren't words, but they communicate volumes. It's fascinating, isn't it? The sheer complexity of just talking.

"""

long_text = "Hello there! I'm excited to hear some of your stories today. When you're ready, we can start with our first story theme."
texts = {
    # 'EN_NEWEST': long_text,  # The newest English base speaker model
    'EN': long_text,
    # 'ES': "El resplandor del sol acaricia las olas, pintando el cielo con una paleta deslumbrante.",
    # 'FR': "La lueur dorée du soleil caresse les vagues, peignant le ciel d'une palette éblouissante.",
    # 'ZH': "在这次vacation中，我们计划去Paris欣赏埃菲尔铁塔和卢浮宫的美景。",
    # 'JP': "彼は毎朝ジョギングをして体を健康に保っています。",
    # 'KR': "안녕하세요! 오늘은 날씨가 정말 좋네요.",
}


src_path = f'{output_dir}/original_nuances.wav'

# Speed is adjustable
speed = 1.0

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'../tts/openvoice_models_downloaded/checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        if torch.backends.mps.is_available() and device == 'cpu':
            torch.backends.mps.is_available = lambda: False
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'./output_v2_nuances_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)


 > Text split to sentences.
Hello there! I'm excited to hear some of your stories today. When you're ready, we can start with our first story theme.
 > ===========================


100%|██████████| 1/1 [00:05<00:00,  5.86s/it]


 > Text split to sentences.
Hello there! I'm excited to hear some of your stories today. When you're ready, we can start with our first story theme.
 > ===========================


100%|██████████| 1/1 [00:03<00:00,  3.23s/it]


 > Text split to sentences.
Hello there! I'm excited to hear some of your stories today. When you're ready, we can start with our first story theme.
 > ===========================


100%|██████████| 1/1 [00:02<00:00,  2.81s/it]


 > Text split to sentences.
Hello there! I'm excited to hear some of your stories today. When you're ready, we can start with our first story theme.
 > ===========================


100%|██████████| 1/1 [00:04<00:00,  4.09s/it]


 > Text split to sentences.
Hello there! I'm excited to hear some of your stories today. When you're ready, we can start with our first story theme.
 > ===========================


100%|██████████| 1/1 [00:03<00:00,  3.34s/it]


In [5]:
from melo.api import TTS

long_text = "Hello there! This is Corian. This is a test to synthesize a voice."
texts = {
    # 'EN_NEWEST': long_text,  # The newest English base speaker model
    'EN': long_text,
    # 'ES': "El resplandor del sol acaricia las olas, pintando el cielo con una paleta deslumbrante.",
    # 'FR': "La lueur dorée du soleil caresse les vagues, peignant le ciel d'une palette éblouissante.",
    # 'ZH': "在这次vacation中，我们计划去Paris欣赏埃菲尔铁塔和卢浮宫的美景。",
    # 'JP': "彼は毎朝ジョギングをして体を健康に保っています。",
    # 'KR': "안녕하세요! 오늘은 날씨가 정말 좋네요.",
}


src_path = f'{output_dir}/Corian Recording.wav'

# Speed is adjustable
speed = 1.0

for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'../tts/openvoice_models_downloaded/checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        if torch.backends.mps.is_available() and device == 'cpu':
            torch.backends.mps.is_available = lambda: False
        model.tts_to_file(text, speaker_id, src_path, speed=speed)
        save_path = f'./output_openvoice_clone_corian.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message)


/opt/homebrew/Caskroom/miniconda/base/envs/MUC_py311new/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


 > Text split to sentences.
Hello there! This is Corian. This is a test to synthesize a voice.
 > ===========================


100%|██████████| 1/1 [00:04<00:00,  4.97s/it]


 > Text split to sentences.
Hello there! This is Corian. This is a test to synthesize a voice.
 > ===========================


100%|██████████| 1/1 [00:04<00:00,  4.47s/it]


...

100%|██████████| 1/1 [00:03<00:00,  3.84s/it]


 > Text split to sentences.
Hello there! This is Corian. This is a test to synthesize a voice.
 > ===========================


100%|██████████| 1/1 [00:04<00:00,  4.31s/it]


 > Text split to sentences.
Hello there! This is Corian. This is a test to synthesize a voice.
 > ===========================


100%|██████████| 1/1 [00:03<00:00,  3.57s/it]


In [22]:
f'../tts/openvoice_models_downloaded/checkpoints_v2/base_speakers/ses/{speaker_key}.pth'

'../tts/openvoice_models_downloaded/checkpoints_v2/base_speakers/ses/en-default.pth'

In [21]:
speaker_ids

{'EN-US': 0, 'EN-BR': 1, 'EN_INDIA': 2, 'EN-AU': 3, 'EN-Default': 4}

In [20]:
!pip show numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Name: numpy
Version: 2.3.1
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2025, NumPy Developers.
 All rights reserved.

 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are
 met:

     * Redistributions of source code must retain the above copyright
        notice, this list of conditions and the following disclaimer.

     * Redistributions in binary form must reproduce the above
        copyright notice, this list of conditions and the following
        disclaimer

In [9]:
model

TTS(
  (model): SynthesizerTrn(
    (enc_p): TextEncoder(
      (emb): Embedding(219, 192)
      (tone_emb): Embedding(16, 192)
      (language_emb): Embedding(10, 192)
      (bert_proj): Conv1d(1024, 192, kernel_size=(1,), stride=(1,))
      (ja_bert_proj): Conv1d(768, 192, kernel_size=(1,), stride=(1,))
      (encoder): Encoder(
        (spk_emb_linear): Linear(in_features=256, out_features=192, bias=True)
        (drop): Dropout(p=0.1, inplace=False)
        (attn_layers): ModuleList(
          (0-5): 6 x MultiHeadAttention(
            (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
            (drop): Dropout(p=0.1, inplace=False)
          )
        )
        (norm_layers_1): ModuleList(
          (0-5): 6 x LayerNorm()
        )
        (ffn_layers): ModuleList(


In [10]:
# TTS(language=language, device=device)
language

'EN'

In [11]:
import os
from pydub import AudioSegment

def convert_m4a_to_wav(m4a_file_path, wav_output_path):
    """
    Converts an M4A audio file to WAV format.

    Args:
        m4a_file_path (str): The path to the input M4A file.
        wav_output_path (str): The desired path for the output WAV file.
    """
    try:
        # Load the M4A file
        sound = AudioSegment.from_file(m4a_file_path, format="m4a")

        # Export as WAV
        sound.export(wav_output_path, format="wav")

        print(f"Successfully converted '{m4a_file_path}' to '{wav_output_path}'")

    except FileNotFoundError:
        print(f"Error: FFmpeg or ffprobe not found. Ensure FFmpeg is installed and in your PATH.")
    except Exception as e:
        print(f"Error converting the file: {e}")

if __name__ == "__main__":
    # --- Configuration ---
    input_m4a_file = "/Users/kexinchong/Permissionless x Cracked Labs Hackathon [1kvKpyEvgewGE].m4a"  # Replace with the actual path to your .m4a file
    output_wav_file = "/Users/kexinchong/Downloads/Permissionless_bounties.wav" # Specify the desired name for the WAV output

    # --- Conversion ---
    convert_m4a_to_wav(input_m4a_file, output_wav_file)


Error: FFmpeg or ffprobe not found. Ensure FFmpeg is installed and in your PATH.


In [3]:
import vosk
import json
import wave

# Load the Vosk model (replace 'vosk_lang' with your model path)
model = vosk.Model("/Users/kexinchong/Downloads/speech/clean/models/vosk-model-en-us-0.42-gigaspeech")
recognizer = vosk.KaldiRecognizer(model, 16000)

# Open the WAV file
wf = wave.open("/Users/kexinchong/Downloads/Permissionless_bounties.wav", "rb") # Use the converted WAV file

# Read and process the audio in chunks
while True:
    data = wf.readframes(4096)
    if not data:
        break
    if recognizer.AcceptWaveform(data):
        result = json.loads(recognizer.Result())
        print(result["text"])

# Get the final result after all audio is processed
result = json.loads(recognizer.FinalResult())
print(result["text"])

wf.close()


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=8
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /Users/kexinchong/Downloads/speech/clean/models/vosk-model-en-us-0.42-gigaspeech/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /Users/kexinchong/Downloads/speech/clean/models/vosk-model-en-us-0.42-gigaspeech/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:294) Loading words from /Users/kexinchong/Downloads/speech/clean/models/vosk-model-en-us-0.42-gigaspeech/graph/words.t









































































































oh

a
a

ah
ah

hi
aw


ah




whoa



















































oh


a
ah
oh oh


or


oh

ah
oh

oh
ah her
ah
oh
wow
oh
ah ooh ahh
oh
ah

ah oh ah oh
oh

oh






























































ah
oh





























oh

a

ah
whoa whoa
oh
ah


a
a little bit
a
ah
ah law literally
ah



































































































ah
oh a
a
a
i
oh
oh
oh
we are
ah
a

a

it'll be a


replace
wow





the
a

wow
ah
wow

wow














































oh


ah


ah
ah
oh
little more
um
no

oh
yeah
a
ah
a
ah
ah
for


oh

a
wow
a

wow

a
a
ah
huh
oh

oh really

ooh
oh

her


ah
oh


oh

oh



ah


uh

yeah
ah

a

oh

ah
wow


oh




a


ah
ah

a oh wow



oh


ah
ugh

ah ah
oh
ah



woo
a











oh
oh
oh
oh


ah







ah
ah

oh





ah


ah
ah
wow
a da
oh
a

LOG (VoskAPI:RebuildRepository():determinize-lattice-pruned.cc:283) Rebuilding repository.


ah
oh
wow

a
a
ah



ah
ah
ah
ah
a

oh


ah
a

a

ah
a



ah

oh
oh
oh
a
uh oh
oh
oh
ah
oh
oh
a
availability
a burden
a little
oh


ah

a ah
oh





one



oh

oh

huh
oh
oh
oh










a
um


oh

a

ooh
ah

ah

ah

ah
oh
a
a

oh wow
oh

ah



a




wow
a little
oh
a
oh

a
ah




ah

ah


huh

ah

oh
ah
ah ah


oh




oh




ah

ah


whatever
a
ah


ah
ah
ah

oh

ah




ah



oh
ah
ah
ah

ah
wow
oh








b
ah
ah

a o


ah

a
oh
ah




oh
a

oh
ah

ah




oh
oh

a
ah

oh


a
a


ah
oh wow
oh
ah

ah
oh oh



ah
ah

a
look
ah
ah
ah
oh

ah


ah

a




ah
ah

ah yeah
a





ah
ah


ah




a
a
a



oh
oh

oh oh
oh


oh

oh



oh


ah




really







a
ah e
a



a
ooh

oh oh

oh
ah
ooh
a
ah
a
ah

a
a
a ruling
e

a
ah
a

a
ah
oh


a ah

oh oh oh
ah
oh
ah
ah
ah

ah
ah



ah
you can
ah

ah
yeah

ah

a

ah ah
ah




oh
a
a


ah
a

i think
oh



ah

oh

oh







ooh
ah ah





yeah



ah

oh

ah
ah

a

oh







ah

oh oh oh


ah



oh
oh

oh
whoa



ah
oh
ah
ah
ah
ah

ah
a
ah
abby


ah


ah

LOG (VoskAPI:RebuildRepository():determinize-lattice-pruned.cc:283) Rebuilding repository.


oh

oh
ah



yeah
a

ah ha

a ah
a
ah
a
a ah
me
ah

a


added
ah
maybe

nah
oh

a

ah in
ah
ah
a o ah
ah
a
ah
oh

a o
a
ah
a ah
a
a



wow
oh

haha

ah a
a
ooh
a
a
huh
a
a
maybe
ah
wow
really


oh



ah
oh
aw
oh oh oh
oh
oh
ah
ah wow
oh
ah whatever
oh oh oh oh wow
oh
wow
oh
oh
oh
oh oh oh oh
oh oh
ah
a little bit
oh
oh
oh
oh wow
oh
oh oh

ah ah ah
ah
ah
ah
ah
ah
ah her

ah
ah
ah
oh
a
ah


oh
oh

a ah the
a a a a
a
a
ah
oh











oh
oh
oh
a

a



a
a
a a

a



yeah




e
wow
whoa
a

me
a


yeah




oh

yeah


oh yeah
oh
yeah


a


a


ah

ah
a
a
ah
a
ah
a


me




yeah
oh wow
e
a more

oh
a

a
ah
and
yeah
ah
e
oh

ah

yeah
oh
a



um
oh
yeah

ooh




oh



KeyboardInterrupt: 

In [ ]:
import os
import re
from fish_audio_sdk import Session, TTSRequest, ReferenceAudio
import logging

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def clone_voice_with_fish(text: str, reference_audio_path: str, output_path: str, reference_text: str = "Text in reference audio"):
    """
    Generates speech with a cloned voice using the Fish Audio API.

    :param text: The text to be converted to speech.
    :param reference_audio_path: Path to the reference audio file for voice cloning.
    :param output_path: Path to save the generated audio file.
    :param reference_text: The transcription of the reference audio. This is important for better quality.
    """
    # Restrict the input text to the first 450 characters.
    if len(text) > 100:
        text = '.'.join(text.split('.')[15:16])[:100]
    
    # Remove special words (e.g., __Gutenberg__, _very_)
    text = re.sub(r'__\w+__\s*|_\w+_\s*', '', text)
    logging.info(f"Text for TTS: {text}")

    # Load the API key from an environment variable for security.
    api_key = os.getenv("FISH_AUDIO_API_KEY")
    if not api_key:
        raise ValueError("FISH_AUDIO_API_KEY environment variable not set.")
    
    session = Session(api_key)

    with open(reference_audio_path, "rb") as audio_file:
        with open(output_path, "wb") as f:
            for chunk in session.tts(TTSRequest(
                text=text,
                backend='s1',
                references=[
                    ReferenceAudio(
                        audio=audio_file.read(),
                        text=reference_text,
                    )
                ]
            )):
                f.write(chunk)
    logging.info(f"File output to: {output_path}")

if __name__ == '__main__':
    # This is an example of how to use the function.
    # You would import clone_voice_with_fish from this file into your app.py.
    clone_voice_with_fish(
        text="Hello there. This is Corian. This is a test to synthesize a voice.",
        reference_audio_path="Corian Recording short.mp3",
        output_path="output_fish_clone_corian_demo2.mp3",
        # It's best to have an accurate transcript of the reference audio for better results.
        reference_text="""And then there are the fillers. You know, like, 'We need to, uhh, re-evaluate our strategy.' That 'uhh' isn't just wasted breath; it often signals that the speaker is searching for the right words, or maybe slightly uncertain.

Sometimes it's a quick intake of breath, a little 'agh' of surprise or mild annoyance. Or a sigh – a deep 'aaah,' conveying relief or perhaps exasperation. It's subtle, but we interpret it instantly.

Even a simple 'oh!' can be, well, varied. Is it 'Oh, really?' with rising surprise, or 'Oh, no...' with a falling, disappointed tone? These tiny vocalizations, the pitch shifts, the slight hesitations, the emphasis on certain syllables """
    )

2025-07-20 08:11:18,563 - INFO - Text for TTS: Hello there. This is Corian. This is a test to synthesize a voice.
2025-07-20 08:11:19,949 - INFO - HTTP Request: POST https://api.fish.audio/v1/tts "HTTP/1.1 200 OK"
2025-07-20 08:11:20,953 - INFO - File output to: output_fish_clone_corian_demo2.mp3


In [3]:
pip install fish_audio_sdk

  Using cached fish_audio_sdk-2025.6.3-py3-none-any.whl.metadata (3.4 kB)
  Using cached ormsgpack-1.10.0-cp311-cp311-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata (43 kB)
  Using cached httpx_ws-0.7.2-py3-none-any.whl.metadata (9.3 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached fish_audio_sdk-2025.6.3-py3-none-any.whl (8.3 kB)
Using cached httpx_ws-0.7.2-py3-none-any.whl (14 kB)
Using cached ormsgpack-1.10.0-cp311-cp311-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl (376 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [fish_audio_sdk]
Note: you may need to restart the kernel to use updated packages.


In [5]:
!export FISH_AUDIO_API_KEY=08bad605af9640c181e83242229fb84a